## Measuring Variance Explained by a PCA Model

This notebook will show how to measure the amount of variance that can be explained by the top $k$ principal components in a Principal Component Analysis (PCA) model. This technique is used to pick the number of lower dimensional space dimensions when performing dimensionality reduction using PCA.

For the purposes of this demonstration, we will use the wine dataset from the UCI Machine Learning Repository, found at https://archive.ics.uci.edu/ml/datasets/Wine. This demo was inspired by Sebastian Raschka's demo found at https://plot.ly/ipython-notebooks/principal-component-analysis/.

Just as there are multiple methods to compute a PCA model, we will show two different ways to measure the percent of explained variance in the model. This percentage is computed from the eigenvalues obtained after the eigendecomposition of the covariance matrix step in PCA. In short, the eigenvectors with the highest associated absolute eigenvalues are those that account for the most variance in the data. As a result, when building the PCA lower-dimensional data, we choose the $k$ principal components with the highest associated absolute eigenvalues, in non-increasing value order. By normalizing the vector of absolute eigenvalues with the L-1 norm, we obtain, for each feature, the percentage of the overall variance expained by that feature. Then, we obtain the percent variance expained by the chosen set of features by suming up the individual percent values for the chosen features. The vector of eigenvalues can also be easily recovered from the sigular values obtained from the Singular Value Decomposition (SVD) of the original centered matrix.

### Data pre-processing

Standardization makes features in the original feature space be compatible with each other with regards to the measurement scale. This is important in many Data Mining and Machine Learning analyses, and especially for the PCA, which aims to preserve variance. If there is significant difference in measurement scales between features (e.g., one feature is measured in mm and all others in m), the transformation will mainly pick up on the variance produced by some of the features and miss out of the more minute differences in the others. 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# read in the dataset
df = pd.read_csv(
    filepath_or_buffer='data/wine.data', 
    header=None, 
    sep=',')
    
    
# extract the vectors from the Pandas data file
X = df.iloc[:,1:].values

# standardise the data
X_std = StandardScaler().fit_transform(X)

Some of the PCA computation methods require that the data be centered, i.e., the mean of all the sample values for the jth feature is subtracted from all the jth feature sample values.

In [ ]:
# subtract the mean vector from each vector in the dataset
means = np.mean(X_std, axis=0)
X_sm = X_std - means

### Algorithm 1: Computing PCA via the covariance matrix

One way to find the principal components is by an eigendecomposition of the covariance matrix $X_{cov} = \frac{1}{n-1} X^TX$, where $X$ is the centered matrix.

In [ ]:
X_cov = X_sm.T.dot(X_sm) / (X_sm.shape[0] - 1)

# Side-note: Numpy has a function for computing the covariance matrix
X_cov2 = np.cov(X_std.T)
print("X_cov == X_cov2: ", np.allclose(X_cov, X_cov2))

# perform the eigendecomposition of the covariance matrix
eig_vals, eig_vecs = np.linalg.eig(X_cov)

What remains now is to pick the eigenvectors (columns in *eig_vecs*) associated with the eigenvalues in *eig_vals* with the highest absolute values. Let's see first the percent variance expained by each eigenvalue-eigenvector pair. To do this, we sort the absolute eigenvalues and transform the values into percentages by performing L-1 normalization. We then perform a prefix-sum operation on the vector of percentages. The resulting vector will show us, in its $j$th dimension, the percent of explained variance in the PCA dimensionality reduction using $j$ dimensions. We will create a function that we can reuse to do this transformation.

In [ ]:
def percvar(v):
    r"""Transform eigen/singular values into percents.
    Return: vector of percents, prefix vector of percents
    """
    # sort values
    s = np.sort(np.abs(v))
    # reverse sorting order
    s = s[::-1]
    # normalize
    s = s/np.sum(s)
    return s, np.cumsum(s)
print("eigenvalues:    ", eig_vals)
pct, pv = percvar(eig_vals)
print("percent values: ", pct)
print("prefix vector:  ", pv)

## Exercise 1
Plot the `pct` and `pv` vectors and observe the general trend of the variance as more and more dimensions are added.

In [ ]:
# plot feature and overall percent variance
%matplotlib inline
import matplotlib.pyplot as plt


Now, given an expected percent variance $p$, we choose the number of features $k$ with at least that percent explained variance value in the vector $pv$, i.e., the first dimension whose value is greater or equal to the desired percent. 

## Exercise 2
Create a function that, given the overall percent varience vector plotted in the previous exercise and the expected percent variance $p$, returns the number of latent space dimensions that account for $p$% variance in the data. Print out the number of dimensions for $p \in \{40, 60, 80, 90, 95\}$.

In [ ]:
def perck(s, p):
    pass
    return len(s)

for p in [40, 60, 80, 90, 95]:
    print("Number of dimensions to account for %d%% of the variance: %d" % (p, perck(pv, p*0.01)))

### Algorithm 2: Computing PCA via the Singular Value Decomposition (SVD)

We can instead compute the PCA trasformation via the SVD of the centered matrix $X = X_{sm}$. However, we will then need to transform the singular values of $X$ into eigenvalues of $X^TX$ before constructing the percent vector. In general, the non-zero singular values of a matrix $X$ are the square roots of the eigenvalues of the square matrix $X^TX$.

In [ ]:
U,s,V = np.linalg.svd(X_sm)
# singular values of X are the square roots of the eigenvalues of the square matrix X^TX
print("singular values:        ", s)
print("eigenvalues:            ", (np.sort(np.abs(eig_vals)))[::-1])
print("scaled singular values: ", (s**2/(X_sm.shape[0]-1)))

Since L-1 normalization is invariant to scaling by a constant factor, we can simply apply the *percvar* function to the squared singular values. The result will be equivalent to the one from Algorithm 1.

**Note:** Applying the same technique directly to singular values does not give the same result. In practice, you should base your choice of $k$ on the absolute eigenvalues, which can be theoretically explained as a measure of latent variance in the feature space.

## Exercise 3
Use the `percvar` function to verify that the analysis applied to squared singular values gives the same results as the one based on the covariance matrix. Additionally, verify that the analysis based on absolute singular values does not provide the same results.